In [1]:
from pathlib import Path

import pandas as pd
import functions.data_acquisition as data_funcs


%cd ..

import src.configuration as config

Path(config.RAW_DATA_DIRECTORY).mkdir(parents=True, exist_ok=True)
Path(config.INTERIM_DATA_DIRECTORY).mkdir(parents=True, exist_ok=True)
Path(config.PROCESSED_DATA_DIRECTORY).mkdir(parents=True, exist_ok=True)

/workspaces/Tab_4_5_aviation_final_project


In [2]:
data_funcs.download_data(config.INCIDENT_DATA_URL, config.RAW_DATA_DIRECTORY, config.RAW_INCIDENTS_MDB_FILE)

In [3]:
table=data_funcs.parse_mdb(config.RAW_INCIDENTS_MDB_FILE, config.RAW_INCIDENTS_CSV_FILE)

In [ ]:
import subprocess
import pandas as pd
import io
import os

mdb_file = "/workspaces/Tab_4_5_aviation_final_project/data/raw/avall.mdb"
output_dir = "/workspaces/Tab_4_5_aviation_final_project/data/processed"

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Step 1: List tables in the MDB file
def list_tables(mdb_file):
    result = subprocess.run(["mdb-tables", "-1", mdb_file], stdout=subprocess.PIPE, text=True)
    tables = result.stdout.strip().split("\n")
    tables = [t.strip() for t in tables if t.strip()]
    print(f"Found tables: {tables}")
    return tables

# Step 2: Export and save each table to CSV
def load_and_save_tables(mdb_file, tables, output_dir):
    dataframes = {}
    for table in tables:
        print(f"Exporting table: {table}")
        result = subprocess.run(["mdb-export", mdb_file, table], stdout=subprocess.PIPE, text=True)
        csv_output = result.stdout
        if csv_output.strip():  # Check if there's any data
            try:
                df = pd.read_csv(io.StringIO(csv_output))
                dataframes[table] = df
                csv_path = os.path.join(output_dir, f"{table}.csv")
                df.to_csv(csv_path, index=False)
                print(f"Saved '{table}' to {csv_path} (shape={df.shape})")
            except Exception as e:
                print(f"Failed to read table {table}: {e}")
        else:
            print(f"Table {table} has no data.")
    return dataframes

# Run the full process
tables = list_tables(mdb_file)
dataframes = load_and_save_tables(mdb_file, tables, output_dir)

In [ ]:
import pandas as pd
import os

# Set directory and files to join
input_dir = "/workspaces/Tab_4_5_aviation_final_project/data/processed"
output_file = "/workspaces/Tab_4_5_aviation_final_project/data/processed/combined_output.csv"

# Define CSV files and the common column to join on
csv_files = [
    ("events", "ev_id"),
    ("aircraft", "ev_id"),
    ("injury", "ev_id")
]
# ^ These are table names without the .csv extension; must exist in input_dir

# Load the first file as the base DataFrame
base_name, join_key = csv_files[0]
base_df = pd.read_csv(os.path.join(input_dir, f"{base_name}.csv"))

# Iteratively join with the other tables
for table_name, join_col in csv_files[1:]:
    df_to_join = pd.read_csv(os.path.join(input_dir, f"{table_name}.csv"))
    print(f"Joining {table_name} on {join_col}")
    base_df = pd.merge(base_df, df_to_join, on=join_col, how="left")  # or "inner", "outer", etc.

# Save combined DataFrame
base_df.to_csv(output_file, index=False)
print(f"Combined DataFrame saved to: {output_file}")